# Cognyx client debug notebook

This notebook is used to debug the Cognyx client used in cognyx automations.
you will need a .env file with a JWT token to run this notebook. You can get this after logging into the cognyx app.

In the cell below, define the bom_id and view_id you want to use (view_id will default to default view if not provided)

In [1]:
import os

from cognyx_bom_sdk.cognyx import CognyxClient

jwt = os.getenv("COGNYX_TOKEN")

bom_id = "5cebf3b2-299a-4253-a365-a2f49d167911"
view_id = "3ae4cda7-f214-4ab6-8822-32ffd7d0d13f"
base_url = "http://localhost:4000"

if not bom_id:
    raise ValueError("bom_id is required")


cognyx = CognyxClient(base_url=base_url, jwt_token=jwt)

bom_data = await cognyx.load_bom_data(bom_id, view_id)

bom_data


{'bom': Bom(id='5cebf3b2-299a-4253-a365-a2f49d167911', name='Complete Kite', reference='KITE-001', description='Fully assembled kite ready for use outdoors', status='BomStatus.DRAFT', instances=[BomInstance(id='b5bb97e5-f4bf-4084-b956-d2912484ef51', name='Kite Sail Assembly', description='Main sail assembly comprising sail material and attachment reinforcements for flight', bom_id='5cebf3b2-299a-4253-a365-a2f49d167911', view_id='3ae4cda7-f214-4ab6-8822-32ffd7d0d13f', parent_id='5cebf3b2-299a-4253-a365-a2f49d167911', reference='KITE-ASM-01', status='not_started', custom_attributes=[Attribute(id='0ca0b4c2-3626-4744-8fd9-3f7b24588fc5', name='Level', description='', slug='level', options=[], type='NUMBER', value=1, attribute_id='baff6456-721a-44c3-af22-cb5a61a3577d')], object_type=ObjectType(id='45fff945-1dce-4086-a8d8-8a50cb12eb43', name='Assembly', description='Assembly entity type.', slug='assembly', base_unit='pc', bom_attributes=[Attribute(id='baff6456-721a-44c3-af22-cb5a61a3577d', na

It will load the bom data and return it.

In [2]:
from cognyx_bom_sdk.cognyx import CognyxClient

cognyx = CognyxClient(base_url=base_url, jwt_token=jwt)

bom_data = await cognyx.load_bom_data(bom_id, view_id)


When the bom is loaded it will then initialize the BomClient

In [3]:
from cognyx_bom_sdk.client import BomClient


def update_callback(update: dict) -> None:
    """Callback function to handle updates to the BOM."""
    print("Update:", update)


client = BomClient(bom_data=bom_data, update_callback=update_callback)

In [4]:
for i in client.list_bom_instances():
  print(client.get_object_reference(i.id))
  print(client.get_object_attribute(i.id, "level"))

id='5c986118-9a49-4ed5-9c1d-3f5e67cbbd05' name='Kite Sail Assembly' description='Main sail assembly comprising sail material and attachment reinforcements for flight' reference='KITE-ASM-01' status='draft' object_type=ObjectType(id='45fff945-1dce-4086-a8d8-8a50cb12eb43', name='Assembly', description='Assembly entity type.', slug=None, base_unit='pc', bom_attributes=[], catalog_attributes=[], parent_id=None, parent=None, display_name='Assembly') object_type_id='45fff945-1dce-4086-a8d8-8a50cb12eb43' attributes=[]
None
id='dd30229a-8eb9-49e1-9576-07b697c38daf' name='4317626-01' description='LIGHT PIPE FRONT LEFT - PURCHASED' reference='4317626' status='published' object_type=ObjectType(id='b960e786-4919-470d-9cfb-5601e187aedb', name='Part', description='Part entity type.', slug=None, base_unit='pc', bom_attributes=[], catalog_attributes=[], parent_id=None, parent=None, display_name='Part') object_type_id='b960e786-4919-470d-9cfb-5601e187aedb' attributes=[]
None
id='a6dd7672-66ed-48cc-8137

ValueError: Object reference not found for instance ID ce763244-bc36-40a7-8c1b-5ef18afcaafe

You can now use the client to interact with the BOM.

In [ ]:
from functools import reduce

from cognyx_bom_sdk.models import BomInstance

mass_attribute = "added mass g"


def get_instance_mass(instance: BomInstance) -> int:
    """Get the mass of a BOM instance.

    Args:
        instance: BOM instance

    Returns:
        The mass of the instance
    """
    instance_mass = client.get_instance_attribute(instance.id, mass_attribute)

    if instance_mass is None:
        return 0

    print(f"instance {instance.name} weighs {instance_mass}g")
    return instance.quantity * instance_mass


reduce(lambda total, instance: total + get_instance_mass(instance), client.list_bom_instances(), 0)

0

In [ ]:
instance = client.get_bom_instance("child 2")

print(instance.id)


2e301eff-50bc-4a4a-bcad-8ee96c39bf7b
